# **Clean Data Checking: Netflix Movies and TV Shows**

**Group Number:** 97  
**Members:**  
Roy Rui #300176548  
Jiayi Ma #300263220
 

# **Dataset I: Netflix Movies and TV Shows Dataset**  
**Author**: Shivam Bansal  
**Ref**: [Source](https://www.kaggle.com/datasets/shivamb/netflix-shows)  
**Shape**: **12 Columns, 8807 Rows**  

## **Description**  
The **Netflix Titles Dataset** contains metadata about TV shows and movies available on Netflix. The dataset provides detailed information such as **title, director, cast, country of production, release year, rating, and duration**. It is widely used for exploratory data analysis (EDA), data cleaning, and recommendation system development.

| Feature         | Description  | Data Type   |
|----------------|--------------|-------------|
| show_id        | Unique identifier for each title | Categorical |
| type           | Type of content (Movie or TV Show) | Categorical |
| title         | Title of the movie or TV show | Categorical |
| director       | Name of the director(s) | Categorical |
| cast          | List of main actors | Categorical |
| country        | Country where the content was produced | Categorical |
| date_added     | Date when the content was added to Netflix | Categorical |
| release_year   | Year of content release | Numerical |
| rating         | Maturity rating (e.g., TV-MA, PG-13) | Categorical |
| duration       | Duration in minutes (movies) or number of seasons (TV shows) | Categorical |
| listed_in      | Genre categories (e.g., Drama, Comedy) | Categorical |
| description    | Brief description of the title | Categorical |

This dataset is useful for **content analysis, trends in streaming media, and audience preferences**, and is particularly valuable for **clean data checking, missing value analysis, and feature engineering**.

---


**General Imports**

In [383]:
# Import necessary libraries
import pandas as pd
import IPython.display as display
from datetime import datetime
from fuzzywuzzy import fuzz, process

# Load the Netflix dataset
df = pd.read_csv('dataset1/netflix_titles.csv')


---

## **1. Data Type Errors**

### **Cell 1 – Description**  
Data type errors occur when column values do not match their expected data types. This can cause:
- Inaccurate calculations 
- Sorting and filtering issues
- Unexpected behaviors 



### **Cell 2 – Parameters for the Checker**

In [384]:
expected_dtypes = {
    "show_id": "object",  # Unique ID, stored as a string
    "type": "object",  # Categorical: Movie/TV Show
    "title": "object",  # Movie/TV Show title, text
    "director": "object",  # Names, text
    "cast": "object",  # List of actors, text
    "country": "object",  # Country names, text
    "date_added": "datetime64[ns]",  # Should be converted to a datetime format
    "release_year": "int64",  # Should be converted before checking
    "rating": "object",  # Categorical text (e.g., PG, TV-MA)
    "duration": "object",  # Contains both text and numbers
    "listed_in": "object",  # Genre categories, text
    "description": "object"  # Movie/TV Show description, text
}


### **Cell 3 – Checker Code**

In [385]:
# Function to check if actual data types match expected data types
def check_data_types(df, expected_dtypes):
    mismatches = {}
    for column, expected_type in expected_dtypes.items():
        actual_type = df[column].dtype
        if str(actual_type) != expected_type:
            mismatches[column] = actual_type
    return mismatches

# Run the data type check
data_type_issues = check_data_types(df, expected_dtypes)
print(data_type_issues)


{'date_added': dtype('O')}


### **Cell 4 – Report of Findings**

#### **Summary of Data Type Errors**
After running the data type validation, the following column(s) were found to have incorrect data types:

| Column Name  | Expected Type   | Actual Type |
|-------------|----------------|------------|
| `date_added` | `datetime64[ns]` | `object` |

#### **Impact of Data Type Errors**
- **`date_added` should be in datetime format** to allow for proper time-based analysis, such as filtering by year or month.
- Since it's currently stored as a string (`object`), any date-related operations (sorting, filtering by date range) may not work correctly.

#### **Next Steps**
- Convert `date_added` to a `datetime64` format to ensure consistency in date-related operations.





---



## **2. Range Errors**  

### **Cell 1 – Description**  
Range errors occur when numerical values fall outside a reasonable or predefined range. These errors can arise due to data entry mistakes, inconsistencies in data sources, or missing value replacements.  


### **Cell 2 – Parameters for the Checker**

In [386]:
# Define expected value ranges for numerical attributes
expected_ranges = {
    "release_year": (1900, 2025),  # Release year should be within this range  
}


### **Cell 3 – Checker Code**

In [387]:
# Ensure the release_year column is of integer type
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')

# Function to check if numerical values fall within the expected range
def check_range_errors(df, expected_ranges):
    out_of_range = {}
    for column, (min_val, max_val) in expected_ranges.items():
        invalid_values = df[(df[column] < min_val) | (df[column] > max_val)][column].tolist()
        if invalid_values:
            out_of_range[column] = invalid_values
    return out_of_range


# Run the range check
range_issues = check_range_errors(df, expected_ranges)
print("Range issues: ")
print(range_issues)


Range issues: 
{}


### **Cell 4 – Report of Findings: Range Errors**

### **Summary of Range Errors**
After running the range validation, **no issues** were found in the dataset.

### **Insights:**
- All numerical values fall within the expected range.
- `release_year` values are correctly constrained within **1900 - 2025**.


---


## **3. Format Errors**

### **Cell 1 – Description**
Format errors occur when categorical values do not follow the expected structure or pattern. 


### **Cell 2 – Parameters for the Checker**


In [388]:
# Define expected date format pattern for 'date_added'
expected_date_pattern = r"^[A-Za-z]+\s\d{1,2},\s\d{4}$"  # e.g., "September 24, 2021"

### **Cell 3 – Checker Code**

In [389]:
def check_date_format_issues(df):
    # Dictionary to store errors
    errors = {}

    
    # Ensure 'date_added' is treated as a string for pattern matching
    df["date_added"] = df["date_added"].astype(str)

    # Identify rows where 'date_added' does NOT match the expected format (excluding NaT and empty values)
    raw_date_values = df[
        (~df["date_added"].str.match(expected_date_pattern, na=True)) &  # Mismatched formats 
        (df["date_added"].str.strip() != "") & # Exclude empty strings
        (~df["date_added"].isna())  # Exclude missing values
    ]

    # Store the incorrect formats in the errors dictionary
    if not raw_date_values.empty:
        errors["date_added_raw"] = raw_date_values[["date_added"]].head(20)

    return errors

# Run the date format consistency check
date_format_issues = check_date_format_issues(df)
print("Date format issues: ")
print(date_format_issues)


Date format issues: 
{'date_added_raw':               date_added
6066                 nan
6079      August 4, 2017
6174                 nan
6177   December 23, 2018
6213   December 15, 2018
6279        July 1, 2017
6304       July 26, 2019
6318        May 26, 2016
6357    November 1, 2019
6361    December 2, 2017
6368      March 15, 2019
6393     October 1, 2019
6451   December 15, 2017
6456        July 1, 2017
6457      August 4, 2017
6460       April 4, 2017
6519   December 28, 2016
6549      March 31, 2018
6560    February 1, 2019
6603     January 1, 2018}


### **Cell 4 – Report of Findings: Format Errors**  

The following column(s) have **incorrectly formatted dates**:

| Column Name  | Unexpected Format Examples |
|-------------|---------------------------|
| `date_added` | `' August 4, 2017'`, `' December 23, 2018'`, `' March 31, 2018'` | -> with additional space in front  

See following rows: 



In [390]:
print(date_format_issues["date_added_raw"]["date_added"].tolist())

['nan', ' August 4, 2017', 'nan', ' December 23, 2018', ' December 15, 2018', ' July 1, 2017', ' July 26, 2019', ' May 26, 2016', ' November 1, 2019', ' December 2, 2017', ' March 15, 2019', ' October 1, 2019', ' December 15, 2017', ' July 1, 2017', ' August 4, 2017', ' April 4, 2017', ' December 28, 2016', ' March 31, 2018', ' February 1, 2019', ' January 1, 2018']


#### **Insights:**

- The `date_added` column contains **inconsistent date formats**, which can cause parsing issues.

#### **Next Steps:**
- Convert all `date_added` values to a standard **datetime format (`YYYY-MM-DD`)**.

---

## **4. Consistency Errors**

### **Cell 1 – Description**  
Consistency errors occur when data values contradict expected logical constraints. These issues can lead to incorrect analysis and misinterpretation of results. Common examples include:  
- **Dates occurring in the future** (e.g., `date_added` should not be later than today's date).  
- **Unrealistic durations** (e.g., a movie duration of less than 0 minutes is likely incorrect).  
 
 ### **Cell 2 – Parameters for the Checker**


In [391]:
# Define expected consistency checks
consistency_checks = {
    "date_added": datetime.today(),  # Should not be in the future
    "duration": 0
}

### **Cell 3 – Checker Code**

In [392]:
# Convert 'date_added' column to datetime
df["date_added"] = pd.to_datetime(df["date_added"], errors='coerce')

# Extract duration values for minutes only
df["duration_mins"] = df["duration"].str.extract(r'(\d+) min')[0].astype(float)

# Function to check for consistency errors
def check_consistency_errors(df, consistency_checks):
    errors = {}

    # Check if 'date_added' contains future dates
    future_dates = df[df["date_added"] > consistency_checks["date_added"]]
    if not future_dates.empty:
        errors["date_added"] = future_dates
        
    # Check if 'duration' for movies is less than 5 minutes
    short_movies = df[(df["duration_mins"] < consistency_checks["duration"]) & df["duration"].str.contains("min", na=False)]
    if not short_movies.empty:
        errors["duration"] = short_movies

    return errors

# Run the consistency check
consistency_issues = check_consistency_errors(df, consistency_checks)
print(consistency_issues)


{}


### **Cell 4 – Report of Findings: Consistency Errors**

The consistency issues were not identified in this dataset.

---

## **5. Uniqueness Errors**

### **Cell 1 – Description**
Uniqueness errors occur when values that are expected to be unique contain duplicates. These errors can cause inconsistencies in data analysis and may indicate data integrity issues.

- **Duplicate `show_id` values**: Each entry should have a unique `show_id`, as it serves as the primary identifier for movies and TV shows.
- **Duplicate `title` values**: While multiple entries may share a title (e.g., different versions or releases), we check for cases where exact duplicates exist.  

 ### **Cell 2 – Parameters for the Checker** ***n/a***  

 ### **Cell 3 – Checker Code**


In [393]:
# Define a function to check uniqueness errors
def check_uniqueness_errors(df):
    errors = {}

    # Check for duplicate show_id values
    duplicate_show_ids = df[df.duplicated(subset=["show_id"], keep=False)]
    if not duplicate_show_ids.empty:
        errors["duplicate_show_id"] = duplicate_show_ids

    # Check for duplicate title values
    duplicate_titles = df[df.duplicated(subset=["title"], keep=False)]
    if not duplicate_titles.empty:
        errors["duplicate_title"] = duplicate_titles

    return errors

# Run the uniqueness check
uniqueness_issues = check_uniqueness_errors(df)

# Print results
print("Uniqueness issues:")
print(uniqueness_issues)


Uniqueness issues:
{}


### **Cell 4 – Report of Findings: Uniqueness Errors**

The Uniqueness issues were not identified in this dataset.

---

## **6. Presence Errors**

### **Cell 1 – Description**
Presence errors occur when required values are missing (NaN). Missing values in critical columns can lead to:

- **Incorrect assumptions** 
- **Issues in data processing** 

### **Cell 2 – Parameters for the Checker**

In [394]:
# Specify required columns (all columns)
required_columns = [
    "show_id", "type", "title", "director", "cast", "country", 
    "date_added", "release_year", "rating", "duration", "listed_in", "description"
]


 ### **Cell 3 – Checker Code**

In [395]:
# Define a function to check presence errors
def check_presence_errors(df, required_columns):
    missing_values = {}

    for column in required_columns:
        missing_entries = df[df[column].isna()]
        if not missing_entries.empty:
            missing_values[column] = missing_entries[[column]]

    return missing_values

# Run the presence check
presence_issues = check_presence_errors(df, required_columns)

# Print results
print("Presence issues (column: missing count):")
for column, count in presence_issues.items():
    print(f"{column}: {len(count)}")  



Presence issues (column: missing count):
director: 2634
cast: 825
country: 831
date_added: 98
rating: 4
duration: 3


### **Cell 4 – Report of Findings: Presence Errors**

The following columns have **missing values**:

| Column Name  | Missing Count |
|-------------|--------------|
| `director`  | 2634 |
| `cast`  | 825 |
| `country`  | 831 |
| `date_added`  | 98 |
| `rating`  | 4 |
| `duration`  | 3 |

### **Insights:**
- The **`director`** column has the highest number of missing values (**2634 records**), which may impact analysis based on directors.
- The **`cast`** and **`country`** columns also have **significant missing values**, which can limit demographic insights.
- The **`date_added`** column has **98 missing values**, which may affect timeline-based analyses.
- The **`rating`** and **`duration`** columns have **minimal missing values**, but they still need verification to ensure proper categorization.

### **Next Steps:**
- Decide how to handle missing values:
  - Fill missing values where appropriate (e.g., impute `country` based on other metadata).
  - Remove records where missing data affects critical analysis.
- Investigate patterns in missing values to understand if they occur in specific types of content.
- Ensure that categorical fields like `rating` and `duration` have a valid default replacement if necessary.

---


## **7. Length Errors**

### **Cell 1 – Description**  
Length errors occur when textual data does not meet the expected length constraints. These issues can affect readability, data completeness, and text-based analysis.  

For this dataset, we check for:  
- **Short descriptions**: `description` values that are **less than 50 characters**.



### **Cell 2 – Parameters for the Checker**  
The `description` column should have a **minimum of 50 characters** to ensure sufficient context is provided.

In [396]:
# Minimum length for the 'description' column
min_length = 50

### **Cell 3 – Checker Code**

In [397]:
# Function to check for length errors
def check_length_errors(df, min_length):
    errors = {}
    
    # Identify descriptions with length less than the minimum required
    short_descriptions = df[df["description"].str.len() < min_length]
    
    if not short_descriptions.empty:
        errors["short_descriptions"] = short_descriptions[["show_id", "title", "description"]]
    
    return errors
# Run the length check
length_issues = check_length_errors(df, min_length)

# Print results
print("Length issues:")
print(length_issues)

Length issues:
{}


### **Cell 4 – Report of Findings: Length Errors**

The Length issues were not identified in this dataset.

---

## **8. Look-up Errors**

### **Cell 1 – Description**  
Look-up errors occur when categorical values in a dataset do not match predefined, expected values from a reference list. These errors can result from typos, outdated classifications, or inconsistent labeling.


**Unexpected categorical values** (e.g.,`type` is either Movie or TV Show , a `rating` value that does not match standard TV/movie classifications).



### **Cell 2 – Parameters for the Checker**  


In [398]:
# Define expected categorical values for formatting
expected_categories = {
    "type": {"Movie", "TV Show"},  # Should be either Movie or TV Show
    "rating": {"G", "TV-G", "PG", "PG-13", "TV-Y", "TV-Y7", "TV-Y7-FV", "TV-PG",
               "TV-14", "R", "NR", "TV-MA", "NC-17", "UR"},  # Expected ratings
}

### **Cell 3 – Checker Code**

In [399]:
def check_categorical_values(df, expected_categories):
    # Check if categorical columns contain unexpected values.
    invalid_categories = {}
    for column, valid_values in expected_categories.items():
        if valid_values:  # Skip columns with None (diverse categories)
            unexpected_values = df[~df[column].isin(valid_values)][column].unique()
            if len(unexpected_values) > 0:
                invalid_categories[column] = unexpected_values
    return invalid_categories

# Run the categorical value check
category_issues = check_categorical_values(df, expected_categories)
print("Category issues: ")
print(category_issues)

Category issues: 
{'rating': array(['74 min', '84 min', '66 min', nan], dtype=object)}


### **Cell 4 – Report of Findings: Look-up Errors**

The following column(s) have **incorrect category values**:

| Column Name  | Unexpected Values |
|-------------|------------------|
| `rating`    | `'74 min', '84 min', '66 min'` |  

See following rows: 

In [400]:
incorrect_ratings = df[df['rating'].isin(['74 min', '84 min', '66 min'])]
display.display(incorrect_ratings)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_mins
5541,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,2017-04-04,2017,74 min,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi...",NaN
5794,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,2016-09-16,2010,84 min,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...,NaN
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,2016-08-15,2015,66 min,NaN,Movies,The comic puts his trademark hilarious/thought...,NaN


#### **Insights:**
- The `rating` column contains **duration values** (e.g., `'74 min'`), which should not be there. This suggests that some records in the `rating` column are **missput**.  

#### **Next Steps:**

- Investigate why **duration values** appear in the `rating` column.
- Clean the `rating` column by ensuring only valid rating categories are included.
---

## **9. Exact Duplicate Errors**

### **Cell 1 – Description**  
Exact duplicate errors occur when multiple rows in the dataset contain **identical values across all columns**. These errors can inflate dataset size, distort analysis, and misrepresent the actual content distribution.

To detect exact duplicates, we check for:
- **Rows that appear more than once** across all columns.

### **Cell 2 – Parameters for the Checker**  ***n/a***

### **Cell 3 – Checker Code** 


In [401]:
# Function to check for exact duplicate errors
def check_exact_duplicates(df):
    duplicates = df[df.duplicated(keep=False)]  # Keep all instances of duplicates
    return duplicates

# Run the duplicate check
exact_duplicate_issues = check_exact_duplicates(df)

# Print results
print(f"Number of exact duplicate rows: {len(exact_duplicate_issues)}")

Number of exact duplicate rows: 0


### **Cell 4 – Report of Findings: Exact Duplicate Errors**

The exact duplicate issues were not identified in this dataset.

---

## **10. Near Duplicate Errors**

### **Cell 1 – Description**  
ENear duplicate errors occur when records are almost identical but contain slight variations, often due to formatting inconsistencies, spelling errors, or missing values in non-critical fields.

For this dataset, we check for:
- Records with identical `date_added` and `description`, but different metadata (e.g., minor spelling differences in `director` or `cast` fields).
- Records where `title` is nearly the same but differs due to small typos or extra spaces.


### **Cell 2 – Parameters for the Checker**  
Near duplicates based on title and release year.

### **Cell 3 – Checker Code** 

In [402]:
# Count occurrences of each (date_added, release_year) pair
duplicate_date_release = (
    df.groupby(["date_added", "description"])
    .filter(lambda x: len(x) > 1)  # Keep only duplicates
    .sort_values(["date_added", "description"])
)


### **Cell 4 – Report of Findings: Duplicate Date and Description Errors**

The following column(s) have **identical `date_added` and `description` values**, suggesting potential duplicates:

| Column Name  | Identical Value Examples |
|-------------|-------------------------|
| `date_added, description` | `'2021-08-23, Paranormal activity at a lush, abandoned property'`, `'2021-09-11, A budding politician has devious plans to rise...'` |

As displayed below:  

In [403]:
# Display results
display.display(duplicate_date_release)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_mins
4522,s4523,Movie,22 July,Paul Greengrass,"Anders Danielsen Lie, Jon Øigarden, Jonas Stra...","Norway, Iceland, United States",2018-10-10,2018,R,144 min,"Dramas, Thrillers","After devastating terror attacks in Norway, a ...",144.0
5965,s5966,Movie,22-Jul,Paul Greengrass,"Anders Danielsen Lie, Jon Øigarden, Jonas Stra...","Norway, Iceland, United States",2018-10-10,2018,R,144 min,"Dramas, Thrillers","After devastating terror attacks in Norway, a ...",144.0
8051,s8052,Movie,Solo: A Star Wars Story,Ron Howard,"Alden Ehrenreich, Woody Harrelson, Emilia Clar...",United States,2019-01-09,2018,PG-13,135 min,"Action & Adventure, Children & Family Movies, ...",A young Han Solo tries to settle an old score ...,135.0
8052,s8053,Movie,Solo: A Star Wars Story (Spanish Version),Ron Howard,"Alden Ehrenreich, Woody Harrelson, Emilia Clar...",United States,2019-01-09,2018,PG-13,135 min,"Action & Adventure, Children & Family Movies, ...",A young Han Solo tries to settle an old score ...,135.0
3996,s3997,TV Show,February 9,NaN,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",NaN,2019-03-20,2018,TV-14,1 Season,"International TV Shows, TV Dramas","As a psychology professor faces Alzheimer's, h...",NaN
5964,s5965,TV Show,Feb-09,NaN,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",NaN,2019-03-20,2018,TV-14,1 Season,"International TV Shows, TV Dramas","As a psychology professor faces Alzheimer's, h...",NaN
3962,s3963,Movie,15 August,Swapnaneel Jayakar,"Rahul Pethe, Mrunmayee Deshpande, Adinath Koth...",India,2019-03-29,2019,TV-14,124 min,"Comedies, Dramas, Independent Movies","On India's Independence Day, a zany mishap in ...",124.0
5966,s5967,Movie,15-Aug,Swapnaneel Jayakar,"Rahul Pethe, Mrunmayee Deshpande, Adinath Koth...",India,2019-03-29,2019,TV-14,124 min,"Comedies, Dramas, Independent Movies","On India's Independence Day, a zany mishap in ...",124.0
3568,s3569,Movie,Game Over (Hindi Version),Ashwin Saravanan,"Taapsee Pannu, Vinodhini, Parvathi T, Ramya Su...",NaN,2019-08-21,2019,TV-MA,98 min,"Horror Movies, International Movies, Thrillers","As a series of murders hit close to home, a vi...",98.0
3569,s3570,Movie,Game Over (Tamil Version),Ashwin Saravanan,"Taapsee Pannu, Vinodhini, Parvathi T, Ramya Su...","India, Turkey",2019-08-21,2019,TV-MA,98 min,"Horror Movies, International Movies, Thrillers","As a series of murders hit close to home, a vi...",98.0


### **Insights:**
- The dataset contains **duplicate records** where both `date_added` and `description` are the same.
- This suggests **redundant entries** that might need **deduplication**.
- Possible reasons include **data ingestion errors, multiple cataloging, or content re-releases.**



### **Next Steps:**
- Perform **record-level deduplication** to keep unique titles.
- Investigate whether these are **intentional re-releases** or **true duplicates**.
- Maintain a **unique identifier** to track distinct records.

---


# **References**  

### **Dataset I: Netflix Movies and TV Shows Dataset**  
**Author**: Shivam Bansal  
**Ref**: [Source](https://www.kaggle.com/datasets/shivamb/netflix-shows)  
**Shape**: **12 Columns, 8807 Rows**  
 
### **Acknowledgments**
- **ChatGPT**: Formatting markdown texts, paraphrasing, grammar checks.  
- **VSCode Copilot & ChatGPT**: Code debugging, commenting, and troubleshooting assistance.  
